# Deciphering Code with Character-Level RNN





## Dataset

In [1]:
import pandas as pd
import numpy as np

from keras.preprocessing.text import Tokenizer
import numpy as np
from keras.preprocessing.sequence import pad_sequences

In [2]:
df = pd.read_csv('./data/AES.csv')
df

,Passwords,ciphertext,key
0,2428031609,b'1092675125059b44fa88',ZDBQ7O9F
1,4531040045,b'1e26ffabe380c3d0c299',NO9EDSU0
2,almigrana1,b'2259ed1d208cca75004d',4XS7R110
3,quiero95,b'16194b66fbca0eee',BMZPZLBH
4,doitnow2,b'fab56cb16f5cd504',19RM3TPZ
...,...,...,...
199995,dime168,b'ee2b9cf4479a64',E68PIJK0
199996,!l0v3k3v!n,b'e56a59f153590322ed74',REKBYL0P
199997,9870tmf05,b'0faaf9b73bc5bc9940',3QJZBLJP
199998,CAMILA,b'a451626f19cb',SOMFMV7Q


In [3]:
df = df.dropna()[:120000]

In [4]:
df['Passwords'].apply(str)
df['ciphertext'].apply(str)
df

,Passwords,ciphertext,key
0,2428031609,b'1092675125059b44fa88',ZDBQ7O9F
1,4531040045,b'1e26ffabe380c3d0c299',NO9EDSU0
2,almigrana1,b'2259ed1d208cca75004d',4XS7R110
3,quiero95,b'16194b66fbca0eee',BMZPZLBH
4,doitnow2,b'fab56cb16f5cd504',19RM3TPZ
...,...,...,...
119995,quaisha123,b'ee52380036e56d18fedf',N2C957EO
119996,7192qween,b'af145da41167c9823e',OPM4TFXK
119997,janet35,b'64e336d210b11e',ARY69F82
119998,velly,b'3bf88e62b3',056LFPUW


## Preprocessing Data

In [5]:
def tokenize(x):
    x_tk = Tokenizer(char_level=True)
    x_tk.fit_on_texts(x)                 

    return x_tk.texts_to_sequences(x), x_tk

def pad(x, length=None):
    if length is None:
        length = max([len(sentence) for sentence in x])
    
    return pad_sequences(x, maxlen=length, padding="post", truncating="post",)

### Preprocess Pipeline

In [6]:
def preprocess(x, y):
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

In [7]:
preproc_code_sentences, preproc_plaintext_sentences, code_tokenizer, plaintext_tokenizer = preprocess(df['Passwords'], df['ciphertext'])

print('Data Preprocessed')

Data Preprocessed


In [8]:
preproc_code_sentences[0]

array([ 7, 17,  7, 15,  5, 13,  3, 19,  5, 12,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0

In [9]:
len(code_tokenizer.word_index)+1

136

In [10]:
len(plaintext_tokenizer.word_index)+1

18

In [11]:
plaintext_tokenizer.word_index

{'b': 1,
 "'": 2,
 'e': 3,
 'f': 4,
 '0': 5,
 '2': 6,
 '5': 7,
 '9': 8,
 'c': 9,
 'a': 10,
 '3': 11,
 '4': 12,
 'd': 13,
 '7': 14,
 '6': 15,
 '1': 16,
 '8': 17}

In [12]:
preproc_code_sentences.shape

(120000, 255)

In [13]:
preproc_plaintext_sentences.shape

(120000, 513, 1)

# LSTM

In [14]:
from keras.layers import GRU, Input, Dense, TimeDistributed, RNN, LSTM
from keras.models import Model, Sequential
from keras.layers import Activation
from tensorflow.keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
import tensorflow


def simple_model(input_shape, output_sequence_length, code_vocab_size, plaintext_vocab_size):
    x = Input(shape=input_shape[1:])   
    seq = LSTM(units= 64, return_sequences = True, activation="tanh", name='Layer1')(x)
    output = TimeDistributed(Dense(units = plaintext_vocab_size, activation='softmax', name='Layer2'))(seq)
    model = Model(inputs = x, outputs = output)
    model.compile(optimizer='adam', loss= sparse_categorical_crossentropy, metrics=['accuracy'])
    model.summary()
    return model

tmp_x = pad(preproc_code_sentences, preproc_plaintext_sentences.shape[1]) 
tmp_x = tmp_x.reshape((-1, preproc_plaintext_sentences.shape[-2], 1))     

In [15]:
tmp_x.shape

(120000, 513, 1)

In [16]:
simple_rnn_model = simple_model(
    tmp_x.shape,
    preproc_plaintext_sentences.shape[1],
    len(code_tokenizer.word_index)+1,
    len(plaintext_tokenizer.word_index)+1)

KeyboardInterrupt: 

In [ ]:
simple_rnn_model.get_layer(name="Layer1").output.shape

In [ ]:
simple_rnn_model.fit(tmp_x, preproc_plaintext_sentences, batch_size=32, epochs=10, validation_split=0.3, use_multiprocessing=True)

In [ ]:
def logits_to_text(logits, tokenizer):
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], plaintext_tokenizer))

In [ ]:
df['Passwords'][0]

# GRU

In [ ]:
def simple_model1(input_shape, output_sequence_length, code_vocab_size, plaintext_vocab_size):
    x = Input(shape=input_shape[1:])   
    seq = GRU(units= 64, return_sequences = True, activation="tanh", name='Layer1')(x)  # output must be batchsize x timesteps x units
    output = TimeDistributed(Dense(units = plaintext_vocab_size, activation='softmax', name='Layer2'))(seq)
    model = Model(inputs = x, outputs = output)
    model.compile(optimizer='adam', loss= sparse_categorical_crossentropy, metrics=['accuracy'])
    model.summary()
    return model

In [ ]:
simple_rnn_model1 = simple_model1(
    tmp_x.shape,
    preproc_plaintext_sentences.shape[1],
    len(code_tokenizer.word_index)+1,
    len(plaintext_tokenizer.word_index)+1)

In [ ]:
simple_rnn_model1.fit(tmp_x, preproc_plaintext_sentences, batch_size=32, epochs=10, validation_split=0.3, use_multiprocessing=True)

In [ ]:
print(logits_to_text(simple_rnn_model1.predict(tmp_x[:1])[0], plaintext_tokenizer))

In [ ]:
df['Passwords'][0]

# SimpleRNN

In [ ]:
from keras.layers import SimpleRNN 
def simple_model2(input_shape, output_sequence_length, code_vocab_size, plaintext_vocab_size):
    x = Input(shape=input_shape[1:])      
    seq = SimpleRNN(units= 64, return_sequences = True, activation="tanh", name='Layer1')(x)   
    output = TimeDistributed(Dense(units = plaintext_vocab_size, activation='softmax', name='Layer2'))(seq)    
    model = Model(inputs = x, outputs = output)    
    model.compile(optimizer='adam', loss= sparse_categorical_crossentropy, metrics=['accuracy'])    
    model.summary()    
    return model

In [ ]:
simple_rnn_model2 = simple_model2(
    tmp_x.shape,
    preproc_plaintext_sentences.shape[1],
    len(code_tokenizer.word_index)+1,
    len(plaintext_tokenizer.word_index)+1)

In [ ]:
simple_rnn_model2.fit(tmp_x, preproc_plaintext_sentences, batch_size=32, epochs=10, validation_split=0.3, use_multiprocessing=True)

In [ ]:
print(logits_to_text(simple_rnn_model2.predict(tmp_x[:1])[0], plaintext_tokenizer))

In [ ]:
df['Passwords'][0]